##### Copyright 2025 Google LLC.

In [21]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Function calling with Python

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

 Function calling lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with. Function calling lets you use functions as tools in generative AI applications, and you can define more than one function within a single request.

This notebook provides code examples to help you get started. The documentation's [quickstart](https://ai.google.dev/gemini-api/docs/function-calling#python) is also a good place to start understanding function calling.

## Setup

### Install dependencies

In [22]:
%pip install -qU 'google-genai>=1.0.0'

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](../quickstarts/Authentication.ipynb) quickstart for an example.

In [23]:
from google import genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

### Choose a model

Function calling should work with all the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) models with the GenAI SDK. It also works with the 1.5 generation of models.

In [24]:
MODEL_ID="gemini-2.5-flash-preview-05-20" # @param ["gemini-2.5-flash-preview-05-20", "gemini-2.5-pro-preview-03-25", "gemini-2.0-flash", "gemini-2.0-flash-lite"] {"allow-input":true, isTemplate: true}

## Setting up Functions as Tools

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`genai.types.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`


**Example: Lighting System Functions**

Here are 3 functions controlling a hypothetical lighting system. Note the docstrings and type hints.

In [25]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")

light_controls = [enable_lights, set_light_color, stop_lights]
instruction = """
  You are a helpful lighting system bot. You can turn
  lights on and off, and you can set the color. Do not perform any
  other tasks.
"""

## Basic Function Calling with Chat

Function calls naturally fit into multi-turn conversations. The Python SDK's `ChatSession (client.chats.create(...))` is ideal for this, as it automatically handles conversation history.

Furthermore, `ChatSession` simplifies function calling execution via its `automatic_function_calling` feature (enabled by default), which will be explored more later. For now, let's see a basic interaction where the model decides to call a function.

In [26]:
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": light_controls,
        "system_instruction": instruction,
        # automatic_function_calling defaults to enabled
    }
)

response = chat.send_message("It's awful dark in here...")

print(response.text)

LIGHTBOT: Lights enabled.
No problem, I can turn the lights on for you.



## Examining Function Calls and Execution History

To understand what happened in the background, you can examine the chat history.

The `Chat.history` property stores a chronological record of the conversation between the user and the Gemini model. You can get the history using `Chat.get_history()`. Each turn in the conversation is represented by a `genai.types.Content` object, which contains the following information:

**Role**: Identifies whether the content originated from the "user" or the "model".

**Parts**: A list of genai.types.Part objects that represent individual components of the message. With a text-only model, these parts can be:

* **Text**: Plain text messages.
* **Function Call (genai.types.FunctionCall)**: A request from the model to execute a specific function with provided arguments.
* **Function Response (genai.types.FunctionResponse)**: The result returned by the user after executing the requested function.


In [27]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

It's awful dark in here...

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={} name='enable_lights' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='enable_lights' response={'result': None} }
--------------------------------------------------------------------------------


###model:

No problem, I can turn the lights on for you.


--------------------------------------------------------------------------------


This history shows the flow:

1. **User**: Sends the message.

2. **Model**: Responds not with text, but with a `FunctionCall` requesting `enable_lights`.

3. **User (SDK)**: The `ChatSession` automatically executes `enable_lights()` because `automatic_function_calling` is enabled. It sends the result back as a `FunctionResponse`.

4. **Model**: Uses the function's result ("Lights enabled.") to formulate the final text response.

## Automatic Function Execution (Python SDK Feature)

As demonstrated above, the `ChatSession` in the Python SDK has a powerful feature called Automatic Function Execution. When enabled (which it is by default), if the model responds with a FunctionCall, the SDK will:

1. Find the corresponding Python function in the provided `tools`.

2. Execute the function with the arguments provided by the model.

3. Send the function's return value back to the model in a `FunctionResponse`.

4. Return only the model's final response (usually text) to your code.

This significantly simplifies the workflow for common use cases.

**Example: Math Operations**

In [28]:
from google.genai import types # Ensure types is imported

def add(a: float, b: float):
    """returns a + b."""
    return a + b

def subtract(a: float, b: float):
    """returns a - b."""
    return a - b

def multiply(a: float, b: float):
    """returns a * b."""
    return a * b

def divide(a: float, b: float):
    """returns a / b."""
    if b == 0:
        return "Cannot divide by zero."
    return a / b

operation_tools = [add, subtract, multiply, divide]

chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": operation_tools,
        "automatic_function_calling": {"disable": False} # Enabled by default
    }
)

response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)

print(response.text)

That would be 2508 mittens in total.


In [29]:
print_history(chat)

###user:

I have 57 cats, each owns 44 mittens, how many mittens is that in total?

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'a': 57, 'b': 44} name='multiply' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='multiply' response={'result': 2508} }
--------------------------------------------------------------------------------


###model:

That would be 2508 mittens in total.

--------------------------------------------------------------------------------


Automatic execution handled the `multiply` call seamlessly.

## Automatic Function Schema Declaration

A key convenience of the Python SDK is its ability to automatically generate the required `FunctionDeclaration` schema from your Python functions. It inspects:

- **Function Name**: (`func.__name__`)

- **Docstring**: Used for the function's description.

- **Parameters**: Names and type annotations (`int`, `str`, `float`, `bool`, `list`, `dict`). Docstrings for parameters (if using specific formats like Google style) can also enhance the description.

- **Return Type Annotation**: Although not strictly used by the model for deciding which function to call, it's good practice.

You generally don't need to create `FunctionDeclaration` objects manually when using Python functions directly as tools.

However, you can generate the schema explicitly using `genai.types.FunctionDeclaration.from_callable` if you need to inspect it, modify it, or use it in scenarios where you don't have the Python function object readily available.

In [30]:
import json

set_color_declaration = types.FunctionDeclaration.from_callable(
    callable = set_light_color,
    client = client
)

print(json.dumps(set_color_declaration.to_json_dict(), indent=4))

{
    "description": "Set the light color. Lights must be enabled for this to work.",
    "name": "set_light_color",
    "parameters": {
        "properties": {
            "rgb_hex": {
                "type": "STRING"
            }
        },
        "required": [
            "rgb_hex"
        ],
        "type": "OBJECT"
    }
}


## Manual function calling

For more control, or if automatic function calling is not available,  you can process [`genai.types.FunctionCall`](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall) requests from the model yourself. This would be the case if:

- You use a `Chat` with the default `"automatic_function_calling": {"disable": True}`.
- You use [`Client.model.generate_content`](https://googleapis.github.io/python-genai/genai.html#genai.types.) (and manage the chat history yourself).

**Example: Movies**

The following example is a rough equivalent of the [function calling single-turn curl sample](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample) in Python. It uses functions that return (mock) movie playtime information, possibly from a hypothetical API:

In [31]:
def find_movies(description: str, location: str):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str):
    """Find theaters based on location and optionally movie title which are currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

theater_functions = [find_movies, find_theaters, get_showtimes]

After using `generate_content()` to ask a question, the model requests a `function_call`:

In [32]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Which theaters in Mountain View, CA show the Barbie movie?",
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)

print(json.dumps(response.candidates[0].content.parts[0].to_json_dict(), indent=4))

{
    "function_call": {
        "args": {
            "location": "Mountain View, CA",
            "movie": "Barbie"
        },
        "name": "find_theaters"
    }
}


Since this is not using a `ChatSession` with automatic function calling, you have to call the function yourself.

A very simple way to do this would be with `if` statements:

```python
if function_call.name == 'find_theaters':
  find_theaters(**function_call.args)
elif ...
```

However, since you already made the `theater_functions` list, this can be simplified to:


In [33]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    # Find the function object from the list based on the function name
    for func in functions:
        if func.__name__ == function_name:
            return func(**function_args)

part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, theater_functions)

print(result)

['Googleplex 16', 'Android Theatre']


Finally, pass the response plus the message history to the next `generate_content()` call to get a final text response from the model. The next code cell is showing on purpose different ways to write down `Content` so you can choose the one that you prefer.

In [34]:
from google.genai import types
# Build the message history
messages = [
    types.Content(
        role="user",
        parts=[
            types.Part(
                text="Which theaters in Mountain View show the Barbie movie?."
            )
        ]
    ),
    types.Content(
        role="model",
        parts=[part]
    ),
    types.Content(
        role="tool",
        parts=[
            types.Part.from_function_response(
                name=part.function_call.name,
                response={"output":result},
            )
        ]
    )
]

# Generate the next response
response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)
print(response.text)

The Barbie movie is currently playing at the Googleplex 16 and Android Theatre in Mountain View.


This demonstrates the manual workflow: call, check, execute, respond, call again.

## Parallel function calls

The Gemini API can call multiple functions in a single turn. This caters for scenarios where there are multiple function calls that can take place independently to complete a task.

First set the tools up. Unlike the movie example above, these functions do not require input from each other to be called so they should be good candidates for parallel calling.

In [35]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

house_fns = [power_disco_ball, start_music, dim_lights]

Now call the model with an instruction that could use all of the specified tools.

In [36]:
# You generally set "mode": "any" to make sure Gemini actually *uses* the given tools.
party_chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": house_fns,
        "tool_config" : {
            "function_calling_config": {
                "mode": "any"
            }
        }
    }
)

# Call the API
response = party_chat.send_message(
    "Turn this place into a party!"
)


print_history(party_chat)

Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%


###user:

Turn this place into a party!

--------------------------------------------------------------------------------


###model:

...

Function call: { id=None args={'power': True} name='power_disco_ball' }
Function call: { id=None args={'bpm': 130, 'loud': True, 'energetic': True} name='start_music' }
Function call: { id=None args={'brightness': 0.3} name='dim_lights' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='power_disco_ball' response={'result': True} }
Function response: { id=None name='start_music' response={'result': 'Never gonna give you up.'} }
Function response: { id=None name='dim_lights' response={'result': True} }
--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'power': True} name='power_disco_ball' }
Function call: { id=None args={'bpm': 130, 'loud': True, 'energetic': True} name='start_music' }
Function call: { id=None args={'brightness': 0.3} name='dim_lights' }
--------------------------------------------------------------------------------


Notice the single model turn contains three FunctionCall parts, which the SDK then executed before getting the final text response.

## Compositional Function Calling
The model can chain function calls across multiple turns, using the result from one call to inform the next. This allows for complex, multi-step reasoning and task completion.

**Example: Finding Specific Movie Showtimes**

Let's reuse the theater_functions and ask a more complex query that requires finding movies first, then potentially theaters, then showtimes.

In [37]:
chat = client.chats.create(
    model = MODEL_ID,
    config = {
        "tools": theater_functions,
    }
)

response = chat.send_message("""
  Find comedy movies playing in Mountain View, CA on 01/01/2025.
  First, find the movie titles.
  Then, find the theaters showing those movies.
  Finally, find the showtimes for each movie at each theater.
"""
)

print(response.text)
print("\n--- History ---")
print_history(chat)

OK. I have found comedy movies playing in Mountain View, CA on 01/01/2025, the theaters they are playing in, and their showtimes. The comedy movies playing on 01/01/2025 in Mountain View, CA are Barbie and Oppenheimer.

Barbie is playing at Googleplex 16 and Android Theatre at 10:00 and 11:00.
Oppenheimer is playing at Googleplex 16 and Android Theatre at 10:00 and 11:00.

--- History ---


###user:


  Find comedy movies playing in Mountain View, CA on 01/01/2025.
  First, find the movie titles.
  Then, find the theaters showing those movies.
  Finally, find the showtimes for each movie at each theater.


--------------------------------------------------------------------------------


###model:

...

###user:

Function response: { id=None name='get_showtimes' response={'result': ['10:00', '11:00']} }
Function response: { id=None name='get_showtimes' response={'result': ['10:00', '11:00']} }
Function response: { id=None name='get_showtimes' response={'result': ['10:00', '11:00']} }
Function response: { id=None name='get_showtimes' response={'result': ['10:00', '11:00']} }
--------------------------------------------------------------------------------


###model:

OK. I have found comedy movies playing in Mountain View, CA on 01/01/2025, the theaters they are playing in, and their showtimes. The comedy movies playing on 01/01/2025 in Mountain View, CA are Barbie and Oppenheimer.

Barbie is playing at Googleplex 16 and Android Theatre at 10:00 and 11:00.
Oppenheimer is playing at Googleplex 16 and Android Theatre at 10:00 and 11:00.

--------------------------------------------------------------------------------


Here you can see that the model made seven calls to answer your question and used the outputs of them in the subsequent calls and in the final answer.

## Function Calling Configuration using Modes

While AUTO mode (or the SDK's default automatic execution) is often sufficient, you can precisely control when and which functions the model is allowed to call using the tool_config parameter during model/chat initialization or in send_message.

The `tool_config` accepts a ToolConfig object, which contains a `FunctionCallingConfig`.

The `FunctionCallingConfig` has two main fields:

- `mode`: Controls the overall function calling behavior (AUTO, ANY, NONE).

- `allowed_function_names`: An optional list of function names the model is restricted to calling in this turn.

### AUTO (Default Mode)

- Behavior: The model decides whether to respond with text or to call one or more functions from the provided `tools`. This is the most flexible mode.

- SDK Default: When using ChatSession with automatic execution enabled, the underlying behavior effectively uses `AUTO` mode unless overridden by `tool_config`.

In [38]:
chat = client.chats.create(model=MODEL_ID)

response = chat.send_message(
    message="Turn on the lights!",
    config={
        "system_instruction": instruction,
        "tools": light_controls,
        "tool_config" : types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(
                mode="auto"
            )
        )
    }
)

print_history(chat)

LIGHTBOT: Lights enabled.


###user:

Turn on the lights!

--------------------------------------------------------------------------------


###model:

...

###user:

Function response: { id=None name='enable_lights' response={'result': None} }
--------------------------------------------------------------------------------


###model:

OK. I have turned on the lights.


--------------------------------------------------------------------------------


### NONE Mode
Behavior: The model is explicitly prohibited from calling any functions, even if tools are provided. It will only respond with text. Useful for turns where you want a purely conversational response.

In [39]:
none_chat = client.chats.create(model=MODEL_ID)

response = none_chat.send_message(
    message="Hello light-bot, what can you do?",
    config={
        "system_instruction": instruction,
        "tools": light_controls, # Tools are provided
        "tool_config" : types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(
                mode="none"
            )
        ) # but NONE mode prevents their use
    }
)

print_history(none_chat)

###user:

Hello light-bot, what can you do?

--------------------------------------------------------------------------------


###model:

Hello! I can help you with your lights. I can:
*   Turn lights on
*   Turn lights off
*   Set the color of the lights

--------------------------------------------------------------------------------


### ANY Mode
- Behavior: Forces the model to call at least one function.

  - If allowed_function_names is set, the model must choose one or more functions from that list.

  - If allowed_function_names is not set, the model must choose one or more functions from the full tools list.

- If automatic function calling is enabled, the SDK will call functions automatically until [maximum_remote_calls](https://googleapis.github.io/python-genai/genai.html#genai.types.AutomaticFunctionCallingConfig.maximum_remote_calls) is reached (default: 10).
- To allow x automatic function calls, set maximum_remote_calls to x + 1. [Read more](https://pypi.org/project/google-genai/#:~:text=Function%20calling%20with%20ANY%20tools%20config%20mode)
- Use Case: Useful when the application state dictates that the next step must involve a specific action or set of actions.

In [40]:
chat = client.chats.create(model=MODEL_ID)

response = chat.send_message(
    "Make this place PURPLE!",
    config={
        "system_instruction": instruction,
        "tools": light_controls, # Provide all tools
        "tool_config" : {
            "function_calling_config": {
                "mode": "any"
            }
        },
        "automatic_function_calling": {
            "maximum_remote_calls" : 1
        }
      } # But restrict to available_fns with ANY mode
)

print_history(chat)

LIGHTBOT: Lights enabled.


###user:

Make this place PURPLE!

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={} name='enable_lights' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='enable_lights' response={'result': None} }
--------------------------------------------------------------------------------


###model:

Function call: { id=None args={} name='enable_lights' }
--------------------------------------------------------------------------------


## Next Steps
### Useful API references:

- The [genai.Client](https://googleapis.github.io/python-genai/genai.html#module-genai.client) class
  - Its [Client.models.generate_content](https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content) method has a [genai.types.GenerateContentConfig](https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig) field that is in particular used to set the tools and function calls.
    - The config's `tools` attribute contains a list of [genai.types.Tools](https://googleapis.github.io/python-genai/genai.html#genai.types.Tool) objects.
    - The `function_declarations` attribute contains a list of [genai.types.FunctionDeclarations](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionDeclaration) objects.
- The [response](https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentResponse)'s [candidate](https://googleapis.github.io/python-genai/genai.html#genai.types.Candidate)'s [content](https://googleapis.github.io/python-genai/genai.html#genai.types.Content)'s [parts](https://googleapis.github.io/python-genai/genai.html#genai.types.Part) may contain a [genai.types.FunctionCall](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall), in `response.candidates[0].contents.parts[0]`.
- if `automatic_function_calling` is not disabled, the [genai.Chats](https://googleapis.github.io/python-genai/genai.html#module-genai.chats) session executes the call, and sends back the [genai.types.FunctionResponse]https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionResponse).
- In response to a [FunctionCall](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall) the model always expects a [FunctionResponse](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionResponse).
- If you reply manually using [Chats.send_message](https://googleapis.github.io/python-genai/genai.html#genai.chats.AsyncChat.send_message) or [models.generate_content](https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content) remember thart the API is stateless you have to send the whole conversation history (a list of [Content](https://googleapis.github.io/python-genai/genai.html#genai.types.Content) objects), not just the last one containing the `FunctionResponse`.

### Related examples

Check out these examples using function calling to give you more ideas on how to use that very useful feature:
* [Barista Bot](../examples/Agents_Function_Calling_Barista_Bot.ipynb), an agent to order coffee
* [Browser-as-a-tool](../examples/Browser_as_a_tool.ipynb), using function calling to call a web-browser.
* Using function calling to [re-rank seach results](../examples/Search_reranking_using_embeddings.ipynb).
* [Using tools with the Live API](../quickstarts/Get_started_LiveAPI_tools.ipynb), using function calling and other tools with the Live APIs.

### Continue your discovery of the Gemini API

Learn how to control how the Gemini API interact with your functions in the [function calling config](../quickstarts/Function_calling_config.ipynb) quickstart, discover how to control the model output in [JSON](../quickstarts/JSON_mode.ipynb) or using an [Enum](../quickstarts/Enum.ipynb) or learn how the Gemini API can generate and run code by itself using [Code execution](../quickstarts/Code_Execution.ipynb)